In this Demo we will be exploring and visualizing Gaussian Processeses

In [1]:
#importing neccesary libraries
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from ipywidgets import *
%matplotlib inline

In [2]:
#defining neccesary functions to be used later
#Demo begins after this
def se_kernel(x1, x2, sigma, l):
    return (sigma**2)*np.exp(-(np.linalg.norm(x1-x2)**2)/(2*(l**2)))
def se_kernel2D(x, y, sigma, l):
    return (sigma**2)*np.exp(-(x**2+y**2)/(2*(l**2)))
def se_cov(x, sigma, l):
    n = np.shape(x)[0]
    cov = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            cov[i][j] = se_kernel(x[i], x[j], sigma, l)
    return cov
def se_mu_posterior(x, sigma, l, f):
    N = np.shape(x)[0]
    K = se_cov(x, sigma, l)
    I = np.eye(N)
    Ki = np.linalg.inv(K+10**(-7)*I)
    mu = np.matmul(np.linalg.inv(Ki + (N/((sigma)**2))*I), f) * (N/((sigma)**2)) 
    return mu
def se_prior_sample(x, sigma, l):
    n = np.shape(x)[0]
    cov = se_cov(x,sigma,l)
    mu = np.zeros(np.shape(x)[0])
    y = np.random.multivariate_normal(np.zeros(n), cov, 1).T
    return np.squeeze(y)
def se_post_cov(x, sigma, l):
    N = np.shape(x)[0]
    K = se_cov(x, sigma, l)
    I = np.eye(N)
    Ki = np.linalg.inv(K+10**(-7)*I)
    Kf = np.linalg.inv(Ki + (N/((sigma)**2))*I)
    return Kf
def se_ppd(x, f ,sigma, l, xp):
    n = np.shape(x)[0]
    m = np.shape(xp)[0]
    mu = np.zeros(m)
    var = np.zeros(m)
    f = np.expand_dims(f, axis = 1)
    for i in range(m):
        kp = np.zeros((n,1))
        K = se_cov(x, sigma, l)
        I = np.eye(n)
        Ki = np.linalg.inv(K+10**(-7)*I)
        for j in range(n):
            kp[j] = se_kernel(x[j],xp[i],sigma,l)
        mu[i] = np.matmul( np.matmul(kp.T, Ki) , f)
        var[i] = se_kernel(xp, xp, sigma, l) - np.matmul( np.matmul(kp.T, Ki) , kp)
    return mu, var
def se_contour(sigma, length):
    x = np.linspace(-4, 4, 30)
    y = np.linspace(-4, 4, 30)    
    X, Y = np.meshgrid(x, y)
    Z = se_kernel2D(X, Y, sigma, length)
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.set_title('surface plot of se for sigma = '+str(sigma)+'and l = '+str(length));
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z');
    ax.set_zlim3d(0, 8)
    ax.plot_surface(X, Y, Z, rstride=1, cstride=1,
                    cmap='viridis', edgecolor='none')
def se_f(sigma, length):
    plt.figure(figsize = (7, 5.5))
    x = np.arange(0,10,0.1)
    y = se_prior_sample(x, sigma, length)
    plt.plot(x, y, linewidth = 4.0)
    plt.ylim(-20, 20)
    plt.title('Sample Drawn from Prior for sigma = ' + str(sigma) + ' and length = ' + str(length))
    plt.show()
def se_cov_plot(length):
    plt.figure(figsize = (7, 5.5))
    x = np.arange(0,10,0.1)
    cov = se_cov(x,1,length)
    plt.imshow(cov, cmap='Blues', interpolation='nearest')
    plt.title('Covariace matrix of prior visualized for length = ' + str(length))
    plt.show()
def se_post_cov_plot(sigma, length):
    plt.figure(figsize = (7, 5.5))
    x = np.arange(0,10,0.1)
    cov = se_post_cov(x,sigma,length)
    plt.imshow(cov, cmap='Blues', interpolation='nearest')
    plt.title('Covariace matrix of prior visualized for length = ' + str(length))
    plt.show()
def se_posterior_plot(sigma, length):
    noise_sigma = 0.2
    s = np.random.normal(0, noise_sigma, 100)
    x = np.linspace(0,10,100)
    y = np.sin(x) + s
    mu = se_mu_posterior(x, sigma, length, y)
    plt.title('Posterior mean based on noisy observations for sigma = ' + str(sigma) + ' and length = ' + str(length))
    plt.plot(x, np.sin(x), linewidth = 3.0)
    plt.plot(x, mu, linewidth = 3.0)
    plt.legend(["Original Curve", "Posterior Mean"])
def se_ppd_plot(sigma, length):
    plt.figure(figsize = (7, 5.5))
    xp = np.linspace(0,10,100)
    x = np.array([1,3,5,7,9])
    f = np.sin(x)
    y, u= se_ppd(x, f ,sigma, length, xp)
    plt.ylim(-1, 1)
    plt.plot(xp, y, linewidth=3.0, label = 'predicted values by ppd')
    std1 = y+2*u
    std2 = y-2*u
    plt.fill_between(xp, std2, std1, alpha=0.5, color='red')
    plt.plot(x, f, 'go', label = 'points of the function given')
    plt.plot(xp, np.sin(xp), 'g', label = 'true function')
    plt.title('PPD for sigma = ' + str(sigma) + ' and length = ' + str(length))
    plt.legend()
    plt.show()

# Exploring the Square Exponential Kernel
The SE kernel is as given below

$ \kappa(x_i,x_j) =  K_{se}(x_1, x_2)  = \sigma^2 exp\left( {-\frac{||x_1-x_2||^2}{2l^2}} \right )$

Visualizing this kernel as we vary the sigma and length (l) parameters we get the graph below

We see that sigma controls the height and the length parameter (l) controls the width or spread of the curve

The Sliders can be used to visualize this change in the kernel

In [3]:
interactive_plot = interactive(se_contour, sigma = (1,3,0.01), length = (0.01,3,0.01))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=2.0, description='sigma', max=3.0, min=1.0, step=0.01), FloatSlider(va…

#GP Prior 

The GP Prior is given by 

$ p(f) \sim \mathcal{N}(0,K) $

Where K is a square matrix with 
$ K_{ij} = \kappa(x_i, x_j)$.

Here $ \kappa(.) $ is a chosen kernel function.

Visualizing $ K $ as we vary length(l) we get the following map for the matrix.

In [4]:
interactive_plot = interactive(se_cov_plot,  length = (0.01,5,0.01))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=2.5, description='length', max=5.0, min=0.01, step=0.01), Output(layou…

We see that for small values of length $ K $ is a square matrix and as we increase length $  K  $ becomes banded with non-zero entries outside of the diagonal of the matrix.

#Samples drawn from prior

The Block below draws samples from the prior for a given sigma and length value which can be adjusted with the slider.

In [5]:
%matplotlib inline
interactive_plot = interactive(se_f, sigma = (0.01,10,0.01), length = (0.01,5,0.01))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=5.0, description='sigma', max=10.0, min=0.01, step=0.01), FloatSlider(…

We see that the sigma parameter controls the height or amplitude of the prior. A higher value of sigma leads to a prior with larger amplitude. The length parameter controls the the frequency or how noisy the prior is. Small value of length leads to a noisy prior and a higher value results in a smoother prior.
 

**Relation Between Prior Covariance Matrix and Prior:**

For small length parameter, the prior covariance matrix is a diagonal matrix implying that the individual points in the prior are independent leading to a noisy distribution with no correlation. For larger values of l we have less noisy prior which is due to covariance between different data points. This is because for higher values of length the Prior Matrix is no longer a diagonal matrix meaning that there is Correlation between data points of the prior making it so that the prior so no longer noisy.

# Posterior for Noisy Observations

Learning the original function from a noisy corrupted version is possible using Gaussian Processes. This can be done by taking a specific GP Likelihood. Assuming that the data is given as $ (X, y) = \{ x_n, y_n \}_{n=1}^{N}$.

  A possible GP Likelihood is

$p(y_n, x_n) = \mathcal{N}(y|f(x_n), \sigma^2) $

The output of the model is assumed to have a variance of $ \sigma^2 $ which is equivalent to a noisy GP.

The Posterior of this model is $  p(f|y) = \mathcal{N}(f | \mu_p, \Sigma_p) $

Where $ \mu_p = (K^{-1}+\frac{N}{\sigma^2}I)^{-1}(\frac{N}{\sigma^2}{f}) $ and $ \Sigma_p = (K^{-1}+\frac{N}{\sigma^2}I)^{-1} $


In the next example a noisy version of the sine function is available and the goal is to obtain the denoised version using gaussian process. This is done by modelling the model using the noisy GP described previously.

**Plot of Posterior Mean:**

Below is a plot of the mean of the posterior of the noisy GP problem described above.

In [6]:
%matplotlib inline
interactive_plot = interactive(se_posterior_plot, sigma = (0.01,10,0.01), length = (0.01,5,0.01))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=5.0, description='sigma', max=10.0, min=0.01, step=0.01), FloatSlider(…

The Posterior Mean is not very sensitive to sigma. However, for very small values os sigma it becomes very noisy.
Similar to what we saw with the prior the length parameter controls how noisy the posterior mean is small l leads to a very noisy posterior mean. Whereas, large values of l lead to a oversmoothened posterior mean which is not able to capture the function appropiately. This is similar to what we observed for the prior.

**Posterior Covariance Matrix**

Plotting the map of the posterior covariance matrix we get the below map

In [7]:
interactive_plot = interactive(se_post_cov_plot, sigma = (0.01,5,0.01),  length = (0.01,5,0.01))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=2.5, description='sigma', max=5.0, min=0.01, step=0.01), FloatSlider(v…

Just as we saw the prior, for small values of the length parameter the Posterior Covariance Matrix is a diagonal matrix and for larger values of the length parameter the matrix is no longer diagonal.

# Estimating the function based on limited information

In this problem we use Gaussian Processes to estimate a function based on a few training data samples of the function that are given.

For this tutorial we choose the sine function and try to estimate it's value using the GP PPD which is given by 

$ p(f_{*}|f) = \mathcal{N}(k_{*}^TK^{-1}f, \kappa( x_{*} , x_{*}) - k_{*}^TK^{-1}k_{*}) $

Along with this the red area enclosed is $ \mu - 2*SD $ to $ \mu + 2*SD $ of the PPD

In [8]:
interactive_plot = interactive(se_ppd_plot, sigma = (0.01,10,0.01), length = (0.01,5,0.01))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=5.0, description='sigma', max=10.0, min=0.01, step=0.01), FloatSlider(…

**Impact of $ \sigma $:**

The impact of $ \sigma $ is primarily on the variance of the predictions of the PPD and only affects the red zone. This implies that to get a good estimate of the confidence of the model the parameter $ \sigma  $ has to be tuned appropriately. Too high value of $ \sigma $ leads to zero standard deviation even when the model is wrong. However, a very high value leads to erroneously large deviation in prediction. 

**Impact of length:**

Small values of the length parameter lead to bad predictions that deviate heavily from the value the PPD should have even at known points. On the other hand for large values of length the ppd is oversmoothed as before and the model fails at both predicting the correct value but also results in the model giving low variance in it's predictions. This means that the model confidently predicts wrong values at these points.